### **Importing Libraries**

In [1]:
import math
import random
from collections import defaultdict
from typing import List
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import deque
import copy
from Card_and_Deck import *
from GameState import GameState
from Game_and_Player import *
import time

### **Helper Functions**

In [2]:
class MCTSNode:
    def __init__(self, game_state, parent=None,move_taken=None):
        self.game_state = game_state  # The game state at this node
        self.parent = parent  # Parent node
        self.children = []  # List of child nodes
        self.visits = 0  # Number of times this node has been visited
        self.total_reward = 0  # Total reward (win/loss)
        self.untried_moves = self.game_state.get_legal_moves()  # Legal moves from this state
        self.move_taken = move_taken # move that led to this game state

    def expand(self,move):
          game_state = self.game_state
          new_game_state = copy.deepcopy(game_state)  # Copy external game state, not self.game_state
          self.untried_moves.remove(move)
          child_game_state = new_game_state.apply_move(move)  # Apply move without modifying original

          child_node = MCTSNode(child_game_state, parent=self, move_taken=move)
          self.children.append(child_node)

          return child_node

    def simulate(self,game_state):

        game_state_new = copy.deepcopy(game_state)
        starting_player_index = game_state.current_player_index
        starting_player_name = game_state.players[starting_player_index].name

        while not game_state_new.is_terminal():
            legal_moves = game_state_new.get_legal_moves()
            if not legal_moves:
                break
            move = random.choice(legal_moves)
            game_state_new.apply_move(move)

        # rewards=game_state_new.tricks_won

        # winner_name = max(rewards, key=rewards.get)

        return game_state_new.tricks_won[starting_player_name]



    def backpropagate(self, reward):
        self.visits += 1
        self.total_reward += reward
        if self.parent:
            self.parent.backpropagate(reward)

    def ucb1(self,child, c=1.4):
        parent_visits = child.parent.visits if child.parent else 1

        if child.visits == 0:
            return float("inf")  # Prioritize unvisited nodes
        return (child.total_reward / child.visits) + c * math.sqrt(math.log(parent_visits) / child.visits)

    def best_child(self, c=1.4):
        node = self
        best = max(node.children, key=lambda child: node.ucb1(child, c))
        return best

    # def tree_policy(self, c=1.4):
    #     """
    #     Traverses the tree from the current node to a leaf node.
    #     Expands the node if there are untried moves.
    #     """

    #     current_node = self
    #     while not current_node.game_state.is_terminal():

    #         if current_node.untried_moves:
    #             return current_node.expand(random.choice(current_node.untried_moves))
    #         else:

    #             current_node = current_node.best_child( c)
    #     return current_node









In [19]:
def mcts_search(root_node, num_simulations=100):

    game_state_new = root_node.game_state
    game_state_copy1 = copy.deepcopy(game_state_new)
    root = MCTSNode(game_state_copy1)  # Root node of the MCTS tree

    for _ in range(num_simulations):
        node = root
        game_state_copy = copy.deepcopy(root.game_state)

        # Tree Traversal
        while node.untried_moves == [] and node.children:
            node = node.best_child(c=1.4)
            game_state_copy.apply_move(node.move_taken)

        # Expansion
        if node.untried_moves:
            move = random.choice(node.untried_moves)
            node = node.expand( move)
            game_state_copy.apply_move(move)

        # Simulation
        reward = node.simulate(game_state_copy)

        # Backpropagation
        node.backpropagate(reward)


    best_child = max(root.children, key=lambda child: child.visits)
    return best_child.move_taken


In [20]:


def evaluate_monte_carlo(player_names,roles,index,num_games=1000, num_simulations=1000):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):
        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0,1,2]

        game = Game(player_names, roles,index)
        game_state = GameState(game)

        # game_state = copy.deepcopy(initial_game_state)




        while not game_state.is_terminal():
            if game_state.current_player_index == 0:  # Monte Carlo player is Player 0
                root_node = MCTSNode(game_state)
                best_move = mcts_search(root_node, num_simulations)
                # print(f"Monte Carlo Best Move: {best_move}")
            else:
                legal_moves = game_state.get_legal_moves()
                best_move = random.choice(legal_moves)

            game_state.apply_move(best_move)

            # print("Table Cards:", game_state.table_cards)
            # print("Current Player Index:", game_state.current_player_index)
            # print("Tricks Won:", game_state.tricks_won)

        rewards=game_state.tricks_won


        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        print(f"Winner: {winner_name}")

        if winner_name == "Alice":
            monte_carlo_wins += 1
            win_results.append(1)
        else:
            win_results.append(0)

    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Rule-Based Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")





In [21]:
# np.random.seed(24)
player_names = ["Alice", "Bob", "Charlie"]
roles = ["Teen", "Do", "Paanch"]
index = [0,1,2]

In [22]:

# np.random.seed(24)
# Initialize players with names and roles
player_names = ["Alice", "Bob", "Charlie"]
roles = ["Teen", "Do", "Paanch"]  # Assign roles to players
index = [0,1,2]

game = Game(player_names, roles,index)

print(f"Trump suit selected: {game.trump_suit}")
print("Starting hands:")

dict_hands={}

for player in game.players:
    dict_hands[player.name]=list(player.hand)
    print(f"{player.name}: {list(player.hand)}")

print(dict_hands)



game_state = GameState(game)


Trump suit selected: Hearts
Starting hands:
Alice: [Q of Clubs, K of Hearts, K of Clubs, 7 of Hearts, 10 of Spades, Q of Spades, Q of Hearts, J of Clubs, 8 of Diamonds, 9 of Diamonds]
Bob: [8 of Clubs, A of Diamonds, 9 of Spades, 7 of Spades, J of Hearts, 9 of Hearts, J of Diamonds, A of Hearts, K of Diamonds, J of Spades]
Charlie: [Q of Diamonds, A of Clubs, 10 of Hearts, 10 of Diamonds, 8 of Spades, 8 of Hearts, 10 of Clubs, K of Spades, A of Spades, 9 of Clubs]
{'Alice': [Q of Clubs, K of Hearts, K of Clubs, 7 of Hearts, 10 of Spades, Q of Spades, Q of Hearts, J of Clubs, 8 of Diamonds, 9 of Diamonds], 'Bob': [8 of Clubs, A of Diamonds, 9 of Spades, 7 of Spades, J of Hearts, 9 of Hearts, J of Diamonds, A of Hearts, K of Diamonds, J of Spades], 'Charlie': [Q of Diamonds, A of Clubs, 10 of Hearts, 10 of Diamonds, 8 of Spades, 8 of Hearts, 10 of Clubs, K of Spades, A of Spades, 9 of Clubs]}


In [29]:

# game_state.apply_move((game_state.get_legal_moves())[0])
# game_state.apply_move((game_state.get_legal_moves())[0])
# game_state.apply_move((game_state.get_legal_moves())[0])
game_state.apply_move((game_state.get_legal_moves())[0])
# print("Initial GameState:")

print(game_state.get_legal_moves())
print(game_state.hands)
print(game_state.table_cards)

[A of Clubs, 10 of Clubs, 9 of Clubs]
{'Alice': [Q of Clubs, K of Hearts, K of Clubs, 7 of Hearts, 10 of Spades, Q of Spades, Q of Hearts, J of Clubs, 9 of Diamonds], 'Bob': [9 of Spades, 7 of Spades, J of Hearts, 9 of Hearts, J of Diamonds, A of Hearts, K of Diamonds, J of Spades], 'Charlie': [A of Clubs, 10 of Hearts, 10 of Diamonds, 8 of Spades, 8 of Hearts, 10 of Clubs, K of Spades, A of Spades, 9 of Clubs]}
[[1, 8 of Clubs]]


In [30]:
root_node = MCTSNode(game_state)
best_move = mcts_search(root_node, num_simulations=1000)
print(f"Monte Carlo Best Move: {best_move}")

Monte Carlo Best Move: A of Clubs


In [31]:
def evaluate_monte_carlo(player_names, roles, index, num_games=1000, num_simulations=1000):
    monte_carlo_wins = 0
    win_results = []

    for _ in range(num_games):

        player_names = ["Alice", "Bob", "Charlie"]
        roles = ["Teen", "Do", "Paanch"]
        index = [0, 1, 2]


        game = Game(player_names, roles, index)
        game_state = GameState(game)


        while not game_state.is_terminal():
            if game_state.current_player_index == 0:  # Monte Carlo player is Player 0
                root_node = MCTSNode(game_state)
                best_move = mcts_search(root_node, num_simulations)
            else:
                legal_moves1 = game_state.get_legal_moves()
                # print(game_state.players[game_state.current_player_index].hand)


                if not legal_moves1:
                    print(f"No legal moves for player {game_state.current_player_index}. Ending game.")
                    break

                best_move = random.choice(legal_moves1)

            game_state.apply_move(best_move)


        rewards = game_state.tricks_won
        max_tricks = max(rewards.values())
        potential_winners = [name for name, count in rewards.items() if count == max_tricks]

        winner_name = potential_winners[0]
        print(f"Winner: {winner_name}")

        if winner_name == "Alice":
            monte_carlo_wins += 1
            win_results.append(1)
        else:
            win_results.append(0)

    # Calculate and print win rate and standard deviation
    win_rate = monte_carlo_wins / num_games
    std_dev = np.std(win_results, ddof=1)

    print(f"Monte Carlo vs Rule-Based Win Rate: {win_rate:.2%}")
    print(f"Standard Deviation of Wins: {std_dev:.4f}")


In [32]:
start=time.time()
evaluate_monte_carlo(player_names,roles,index,num_games=100, num_simulations=100)
end=time.time()
print(f"Execution Time: {end - start:.4f} seconds")

Winner: Bob
Winner: Bob
Winner: Charlie
Winner: Charlie
Winner: Alice
Winner: Bob
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Bob
Winner: Bob
Winner: Alice
Winner: Bob
Winner: Charlie
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Bob
Winner: Charlie
Winner: Bob
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Bob
Winner: Bob
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Charlie
Winner: Charlie
Winner: Bob
Winner: Alice
Winner: Bob
Winner: Alice
Winner: Charlie
Winner: Charlie
Winner: Bob
Winner: Alice
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Bob
Winner: Alice
Winner: Bob
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Alice
Winner: Alice
Winner: Bob
Winner: Bob
Winner: Charlie
Winner: Bob
Winner: Bob
Winner: Bob
Winner: Bob
Winner: Charlie
Winner: Charlie
Winner: Bob
Winner: Bob
Winner: Alice
Winner: Charlie
Winner: Alice
Winner: Bob
Winner: 